# Arithmetic

An important function of pandas is the arithmetic behaviour for objects with different indices. When adding objects, if the index pairs are not equal, the corresponding index in the result will be the union of the index pairs. For users with database experience, this is comparable to an automatic [outer join](https://en.wikipedia.org/wiki/Join_(SQL)#Outer_join) on the index labels. Let’s look at an example:

In [1]:
import numpy as np
import pandas as pd

s1 = pd.Series(np.random.randn(5))
s2 = pd.Series(np.random.randn(7))

If you add these values, you get:

In [2]:
s1 + s2

0   -0.117652
1    1.843403
2   -1.554283
3   -0.052506
4   -0.248629
5         NaN
6         NaN
dtype: float64

The internal data matching leads to missing values at the points of the labels that do not overlap. Missing values are then passed on in further arithmetic calculations.

For DataFrames, alignment is performed for both rows and columns:

In [3]:
df1 = pd.DataFrame(np.random.randn(5,3))
df2 = pd.DataFrame(np.random.randn(7,2))

When the two DataFrames are added together, the result is a DataFrame whose index and columns are the unions of those in each of the DataFrames above:

In [4]:
df1 + df2

,0,1,2
0,-1.617414,1.480734,NaN
1,1.060217,2.245327,NaN
2,0.604860,-1.533916,NaN
3,-0.487990,-1.244289,NaN
4,-2.641149,-1.044312,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Since column 2 does not appear in both DataFrame objects, its values appear as missing in the result. The same applies to the rows whose labels do not appear in both objects.

## Arithmetic methods with fill values

In arithmetic operations between differently indexed objects, a special value (e.g. `0`) can be useful if an axis label is found in one object but not in the other.  The `add` method can pass the `fill_value` argument:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,NaN
6,1.864685,1.405582,NaN


In the following example, we set the two remaining NaN values to `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,0.000000
6,1.864685,1.405582,0.000000


## Arithmetic methods

Method | Description
:----- | :----------
`add`, `radd` | methods for addition (`+`)
`sub`, `rsub` | methods for subtraction (`-`)
`div`, `rdiv` | methods for division (`/`)
`floordiv`, `rfloordiv` | methods for floor division (`//`)
`mul`, `rmul` | methods for multiplication (`*`)
`pow`, `rpow` | methods for exponentiation (`**`)

`r` (English: _reverse_) reverses the method.

## Operations between DataFrame and Series

As with NumPy arrays of different dimensions, the arithmetic between DataFrame and Series is also defined.

In [8]:
s1 + df12

,0,1,2,3,4
0,-3.413133,2.531767,-0.858270,NaN,NaN
1,-0.735503,3.296360,-1.001270,NaN,NaN
2,-1.190859,-0.482883,-1.138355,NaN,NaN
3,-2.283709,-0.193256,-1.718093,NaN,NaN
4,-4.436868,0.006721,0.484069,NaN,NaN
5,-2.451955,-0.862252,-0.513864,NaN,NaN
6,0.068966,2.456614,-0.513864,NaN,NaN


If we add `s1` with `df12`, the additon is done once for each line. This is called _broadcasting_. By default, the arithmetic between the DataFrame and the series corresponds to the index of the series in the columns of the DataFrame, with the rows being broadcast down.

If an index value is found neither in the columns of the DataFrame nor in the index of the series, the objects are re-indexed to form the union:

If instead you want to transfer the columns and match the rows, you must use one of the arithmetic methods, for example:

In [9]:
df12.add(s2, axis='index')

,0,1,2
0,0.060653,3.158801,1.333661
1,1.852586,3.037697,0.304963
2,-0.435558,-2.574335,-1.664910
3,0.723141,-0.033158,0.006902
4,-2.536814,-0.939976,1.102269
5,-1.370872,-2.627920,-0.714636
6,0.512718,0.053615,-1.351967


The axis number you pass is the axis to be aligned to. In this case, the row index of the DataFrame (`axis='index'` or `axis=0`) is to be adjusted and transmitted.

## Function application and mapping

`numpy.ufunc` (element-wise array methods) also work with pandas objects:

In [10]:
np.abs(df12)

,0,1,2
0,1.617414,1.480734,0.344406
1,1.060217,2.245327,0.487406
2,0.604860,1.533916,0.624491
3,0.487990,1.244289,1.204230
4,2.641149,1.044312,0.997933
5,0.656236,1.913285,0.000000
6,1.864685,1.405582,0.000000


Another common operation is to apply a function to one-dimensional arrays on each column or row. The [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method does just that:

In [11]:
df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,0.000000
6,1.864685,1.405582,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    4.505834
1    4.158612
2    2.202163
dtype: float64

Here the function `f`, which calculates the difference between the maximum and minimum of a row, is called once for each column of the frame. The result is a row with the columns of the frame as index.

If you pass `axis='columns'` to `apply`, the function will be called once per line instead:

In [13]:
df12.apply(f, axis='columns')

0    3.098148
1    2.732734
2    2.138776
3    0.756299
4    3.639082
5    1.913285
6    1.864685
dtype: float64

Many of the most common array statistics (such as `sum` and `mean`) are DataFrame methods, so the use of `apply` is not necessary.

The function passed to apply does not have to return a single value; it can also return a series with multiple values:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

df12.apply(f)

,0,1,2
min,-2.641149,-1.913285,-1.204230
max,1.864685,2.245327,0.997933


You can also use element-wise Python functions. Suppose you want to calculate a formatted string from each floating point value in `df12`. You can do this with [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html):

In [15]:
f = lambda x: '%.2f' % x

df12.applymap(f)

,0,1,2
0,-1.62,1.48,-0.34
1,1.06,2.25,-0.49
2,0.60,-1.53,-0.62
3,-0.49,-1.24,-1.20
4,-2.64,-1.04,1.00
5,-0.66,-1.91,0.00
6,1.86,1.41,0.00


The reason for the name `applymap` is that Series has a `map` method for applying an element-wise function:

In [16]:
df12[2].map(f)

0    -0.34
1    -0.49
2    -0.62
3    -1.20
4     1.00
5     0.00
6     0.00
Name: 2, dtype: object